In [1]:
import pandas as pd
import numpy as np

In [3]:
#url = "https://docs.google.com/spreadsheets/d/1TOHrzIpiJQkHLJEPW62l-KtEmOvX698jh7nHfLSUzdg/export?format=csv"
df = pd.read_csv("tw.csv")
df.head()

,項目別,國中,高級中等,專科,大學,年增率,研究所,年增率_1
0,NaN,NaN,（高中、高職）,NaN,NaN,(%),NaN,(%)
1,全體,28,29,32,34.0,5.1,52,4.3
2,性別,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,男性,28,30,32,34.0,5,54,3.8
4,女性,29,29,32,34.0,5.1,48,5.2


In [ ]:
# Remove specific rows
df = df.drop(index=[0, 2, 5, 6, 9])
# Reset index
df = df.reset_index(drop=True)
# Drop year-on-year growth rate columns
df = df.drop(columns=["年增率", "年增率_1"], errors="ignore")
# Replace '---' or '-' with NaN
df = df.replace(['---', '--', '-', '—'], np.nan)
# Convert numeric columns to float
for col in ["國中", "高級中等", "專科", "大學", "研究所"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [8]:
df

,Category,Junior High,Senior High,Junior College,University,Graduate School
0,All Employees,28.0,29.0,32.0,34.0,52.0
1,Male,28.0,30.0,32.0,34.0,54.0
2,Female,29.0,29.0,32.0,34.0,48.0
3,Small & Medium Enterprises,28.0,29.0,30.0,31.0,42.0
4,Large Enterprises & Others,30.0,31.0,35.0,38.0,55.0
5,"Agriculture, Forestry, Fishing & Animal Husbandry",NaN,28.0,NaN,31.0,33.0
6,Mining & Quarrying,NaN,NaN,NaN,33.0,NaN
7,Manufacturing,28.0,29.0,31.0,34.0,58.0
8,Electricity & Gas Supply,NaN,31.0,NaN,43.0,43.0
9,Water Supply & Waste Management,NaN,30.0,NaN,32.0,38.0


In [9]:
# Rename columns from Chinese to English
df = df.rename(columns={
    "項目別": "Category",
    "國中": "Junior High",
    "高級中等": "Senior High",
    "專科": "Junior College",
    "大學": "University",
    "研究所": "Graduate School"
})


In [10]:
# Chinese to English industry mapping
industry_map = {
    "全體": "All Employees",
    "男性": "Male",
    "女性": "Female",
    "中小企業": "Small & Medium Enterprises",
    "大企業及其他": "Large Enterprises & Others",
    "農、林、漁、牧業": "Agriculture, Forestry, Fishing & Animal Husbandry",
    "礦業及土石採取業": "Mining & Quarrying",
    "製造業": "Manufacturing",
    "電力及燃氣供應業": "Electricity & Gas Supply",
    "用水供應及污染整治業": "Water Supply & Waste Management",
    "營建工程業": "Construction",
    "批發及零售業": "Wholesale & Retail Trade",
    "運輸及倉儲業": "Transportation & Storage",
    "住宿及餐飲業": "Accommodation & Food Service",
    "出版影音及資通訊業": "Publishing, Audio-Visual & Information Communication",
    "金融及保險業": "Finance & Insurance",
    "不動產業": "Real Estate",
    "專業、科學及技術服務業": "Professional, Scientific & Technical Services",
    "支援服務業": "Administrative & Support Services",
    "公共行政及國防；強制性社會安全": "Public Administration & Defense; Compulsory Social Security",
    "教育業": "Education",
    "醫療保健及社會工作服務業": "Health Care & Social Work",
    "藝術、娛樂及休閒服務業": "Arts, Entertainment & Recreation",
    "其他服務業": "Other Services"
}

# Replace Chinese category names with English
df["Category"] = df["Category"].replace(industry_map)


In [11]:
# Add 'Category_Type' column based on 'Category'
def get_category_type(category):
    if category == "All Employees":
        return "Total"
    elif category in ["Male", "Female"]:
        return "Gender"
    elif category in ["Small & Medium Enterprises", "Large Enterprises & Others"]:
        return "Enterprise Size"
    else:
        return "Industry"

df['Category_Type'] = df['Category'].apply(get_category_type)

# Reorder columns to place 'Category_Type' after 'Category'
cols = df.columns.tolist()
cols.insert(cols.index('Category') + 1, cols.pop(cols.index('Category_Type')))
df = df[cols]

In [12]:
# Exchange rate setting (1 USD = 32.0 TWD, based on Taiwan's recent average exchange rate)
twd_to_usd = 1 / 32.0
for col in ["Junior High", "Senior High", "Junior College", "University", "Graduate School"]:
    df[col] = pd.to_numeric(df[col], errors="coerce") * 1000 * twd_to_usd


In [ ]:
# Export wide DataFrame to CSV
df.to_csv('df_tw_wide.csv', index=False)

df_tw_wide.csv has been saved successfully


In [15]:
# Convert wide DataFrame to long DataFrame
df_long = df.melt(
    id_vars=["Category", "Category_Type"],
    var_name="Education",
    value_name="Salary"
)

# Display long DataFrame
display(df_long.head())


,Category,Category_Type,Education,Salary
0,All Employees,Total,Junior High,875.00
1,Male,Gender,Junior High,875.00
2,Female,Gender,Junior High,906.25
3,Small & Medium Enterprises,Enterprise Size,Junior High,875.00
4,Large Enterprises & Others,Enterprise Size,Junior High,937.50


In [16]:
ppp_conversion_factor = 13.66
twd_exchange_rate = 1 / twd_to_usd # This is 32.0
df_long['Real Salary'] = (df_long['Salary'] * twd_exchange_rate) / ppp_conversion_factor

display(df_long.head())

,Category,Category_Type,Education,Salary,Real Salary
0,All Employees,Total,Junior High,875.00,2049.780381
1,Male,Gender,Junior High,875.00,2049.780381
2,Female,Gender,Junior High,906.25,2122.986823
3,Small & Medium Enterprises,Enterprise Size,Junior High,875.00,2049.780381
4,Large Enterprises & Others,Enterprise Size,Junior High,937.50,2196.193265


In [17]:
numeric_cols = ["Junior High", "Senior High", "Junior College", "University", "Graduate School"]
df_real = df.copy()
df_real[numeric_cols] = df_real[numeric_cols] * twd_exchange_rate / ppp_conversion_factor
df_real

,Category,Category_Type,Junior High,Senior High,Junior College,University,Graduate School
0,All Employees,Total,2049.780381,2122.986823,2342.606149,2489.019034,3806.734993
1,Male,Gender,2049.780381,2196.193265,2342.606149,2489.019034,3953.147877
2,Female,Gender,2122.986823,2122.986823,2342.606149,2489.019034,3513.909224
3,Small & Medium Enterprises,Enterprise Size,2049.780381,2122.986823,2196.193265,2269.399707,3074.670571
4,Large Enterprises & Others,Enterprise Size,2196.193265,2269.399707,2562.225476,2781.844802,4026.354319
5,"Agriculture, Forestry, Fishing & Animal Husbandry",Industry,NaN,2049.780381,NaN,2269.399707,2415.812592
6,Mining & Quarrying,Industry,NaN,NaN,NaN,2415.812592,NaN
7,Manufacturing,Industry,2049.780381,2122.986823,2269.399707,2489.019034,4245.973646
8,Electricity & Gas Supply,Industry,NaN,2269.399707,NaN,3147.877013,3147.877013
9,Water Supply & Waste Management,Industry,NaN,2196.193265,NaN,2342.606149,2781.844802


In [18]:
# Export long DataFrame to CSV
df_long.to_csv('df_tw_long.csv', index=False)